In [1]:
import warnings
warnings.filterwarnings('ignore')
import keras
import sys 

from keras_preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Activation, Conv2D, Dense, Dropout, Flatten, MaxPooling2D, BatchNormalization

Using TensorFlow backend.


In [3]:
model = Sequential()
model.add(Conv2D(20, (5,5), activation='relu', input_shape=(128,128,3)))
model.add(BatchNormalization(momentum=0.9))
model.add(MaxPooling2D())
model.add(Conv2D(50, (5,5), activation='relu'))
model.add(BatchNormalization(momentum=0.9))
model.add(MaxPooling2D())
model.add(Conv2D(100, (4,4), activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(200, (4,4), activation='relu'))
model.add(MaxPooling2D())
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(500))
model.add(Dropout(rate=0.25))
model.add(Dense(8))
model.add(Activation('softmax'))


In [60]:
for layer in model.layers:
    print(layer.output_shape)

(None, 124, 124, 20)
(None, 124, 124, 20)
(None, 62, 62, 20)
(None, 58, 58, 50)
(None, 58, 58, 50)
(None, 29, 29, 50)
(None, 26, 26, 100)
(None, 13, 13, 100)
(None, 10, 10, 200)
(None, 5, 5, 200)
(None, 5, 5, 200)
(None, 5000)
(None, 500)
(None, 500)
(None, 8)
(None, 8)


In [5]:
datagen = ImageDataGenerator(rotation_range=30, 
                             width_shift_range=0.2,
                             height_shift_range=0.2,
                             horizontal_flip=True,
                             vertical_flip=True
                            )

if 'win32' in sys.platform:
    data_directory='F:\\Segmentation_Data\\'
    full_data_directory = data_directory + 'Labelled_imgs\\'

In [6]:
complete_generator = datagen.flow_from_directory(full_data_directory, target_size=(128, 128), color_mode="rgb")

Found 29267 images belonging to 8 classes.


In [8]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(complete_generator, verbose=1, steps_per_epoch=1000, epochs=10)

Epoch 1/10
1000/1000 [==============================] - 160s 160ms/step - loss: 1.1201 - acc: 0.5864
Epoch 2/10
1000/1000 [==============================] - 103s 103ms/step - loss: 0.9295 - acc: 0.6474
Epoch 3/10
1000/1000 [==============================] - 104s 104ms/step - loss: 0.8264 - acc: 0.6866
Epoch 4/10
1000/1000 [==============================] - 104s 104ms/step - loss: 0.7548 - acc: 0.7115s - loss: 0.754
Epoch 5/10
1000/1000 [==============================] - 105s 105ms/step - loss: 0.7039 - acc: 0.7299
Epoch 6/10
1000/1000 [==============================] - 104s 104ms/step - loss: 0.6768 - acc: 0.7449s - ETA: 2s - loss: 
Epoch 7/10
1000/1000 [==============================] - 105s 105ms/step - loss: 0.6452 - acc: 0.7539s - loss: 0.6456 - acc:
Epoch 8/10
1000/1000 [==============================] - 105s 105ms/step - loss: 0.6181 - acc: 0.7653
Epoch 9/10
1000/1000 [==============================] - 103s 103ms/step - loss: 0.6075 - acc: 0.7698
Epoch 10/10
1000/1000 [==========